In [50]:
##########################################################################################
## evaluate detection result
##########################################################################################
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random

GENE_IMG_DIR = '/media/yyp/DATA/EVALUATE/JPEGImages/'
GENE_XML_DIR = '/media/yyp/DATA/EVALUATE/Annotations/'
os.system('rm -rf ' + GENE_IMG_DIR)
os.system('rm -rf ' + GENE_XML_DIR)
os.system('mkdir -p ' + GENE_IMG_DIR)
os.system('mkdir -p ' + GENE_XML_DIR)
time_str = (datetime.datetime.now()).strftime("%Y%m%d")
#f = open("/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/2007_val_ncjnperson.txt", 'r')
f = open("/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/2007_eval.txt", 'r')
data = f.readlines()
seed_xml_names = []
#generated_xml_dir = '/data/darknet/python/' + time_str + '_detection_result/'

for line in data:
    #print line
    line = line[:-1]
    #print(line)
    xml_name = line.replace('JPEGImages', 'Annotations')
    xml_name = xml_name.replace('.jpg', '.xml')
    seed_xml_names.append(xml_name)
seed_xml_names.sort()

# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2])
    #print('w={0};h={1}'.format(W,H))
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    bbox = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
    return bbox

def get_obj_from_xml(xml_name):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    print(img_obj.shape)
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def find_max_iou(obj, objs):
    bbox = get_bbox_from_xmlobj(obj)
    iou_max = 0
    iou_max_obj = False
    for obj_ in objs:
        bbox_ = get_bbox_from_xmlobj(obj_)
        iou_new = IOU(bbox, bbox_)
        
        if(iou_new > iou_max):
            iou_max = iou_new
            iou_max_obj = obj_
    return iou_max_obj, iou_max
            

def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def generate_union_bbox(matrix1, matrix2): #xin,xmax,ymin,ymax
    img_obj_shape = (int(max(matrix1[3], matrix2[3])-min(matrix1[2], matrix2[2])), 
                int(max(matrix1[1], matrix2[1])-min(matrix1[0], matrix2[0])))
    return img_obj_shape

def generate_union_bbox_shape(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    img_obj_shape = (int(max(matrix_array[:,3]) - min(matrix_array[:,2])),
                     int(max(matrix_array[:,1]) - min(matrix_array[:,0])))
    return img_obj_shape

def get_union_bbox_left_top(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    left_top = (int(min(matrix_array[:,0])),
                     int(min(matrix_array[:,2])))
    return left_top
def generate_union_bbox_(matrix_list):
    img_obj_shape = generate_union_bbox_shape(matrix_list)
    left_top = get_union_bbox_left_top(matrix_list)
    union_bbox = [int(left_top[0]), 
                  int(left_top[0]+int(img_obj_shape[1])),
                  int(left_top[1]),
                  int(left_top[1]+int(img_obj_shape[0]))]
    return union_bbox

def generate_new_bbox_within_distance(img_size, img_obj_size, old_bbox, scale=2.5):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    #array_y = np.arange(int(img_size[0] - img_obj_size[0])/3, int(img_size[0] - img_obj_size[0])/3*2)
    array_y = np.arange(max(0, int((old_bbox[3]+old_bbox[2])/2 - scale*img_obj_size[0])),
                        min(int((old_bbox[3]+old_bbox[2])/2 + scale*img_obj_size[0]), int(img_size[0] - img_obj_size[0])))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox

def check_before_insert(img_size, img_obj_size, etree, union_bbox):
    new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 100):
            return (False, False)
        new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    return (new_bbox[0], new_bbox[2])
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def do_inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree, insert_point):
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, insert_point, obj_element, etree)    
    return new_pil_img, new_xml_etree

def inset_dobule_obj_to_an_image_and_xml(img_obj1, img_obj2, img, obj_element1, obj_element2, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0):
            return False
    return True
def caculate_move_vector(bbox, new_point):
    original_xmin, original_ymin = bbox[0], bbox[2]
    diffx, diffy = new_point[0] - original_xmin, new_point[1] - original_ymin
    return (diffx, diffy)

def rotate_img(img, thealta):
    (h_, w_) = img.shape[:2]
    point_list = [(0, 0), (0, h_), (w_, h_), (w_, 0)]
    center = (w_ // 2, h_ // 2)
    roted_point_list = [((point[0]-center[0])*np.cos(np.pi*thealta/180) 
                         - (point[1]-center[1])*np.sin(np.pi*thealta/180) 
                         + center[0], (point[0]-center[0])*np.sin(np.pi*thealta/180) 
                         + (point[1]-center[1])*np.cos(np.pi*thealta/180) + center[1]) for point in point_list]
    #print(roted_point_list)

    temp = np.zeros((2, 4))
    temp[0] = [roted_point[0] for roted_point in roted_point_list]
    temp[1] = [roted_point[1] for roted_point in roted_point_list]
    (xmin, xmax, ymin, ymax) = (np.min(temp[0]), np.max(temp[0]), np.min(temp[1]), np.max(temp[1]))
    print('xmin: {0}, xmax: {1}, ymin: {2}, ymax: {3}'.format(xmin, xmax, ymin, ymax))

    roted_h, roted_w = ymax - ymin, xmax - xmin
    print('roted_h: {0}, roted_w: {1}'.format(roted_w, roted_h))
    
    top_bottom, left_right = int((roted_h - h_)/2), int((roted_w - w_)/2)
    print('top_bottom:{0}, left_right_:{1}'.format(top_bottom, left_right))
    padding = lambda arg : max(arg, 0)
    #dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
    #padding(left_right), cv2.BORDER_CONSTANT)
    dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
                             padding(left_right), cv2.BORDER_CONSTANT)
    
    (h, w) = dst.shape[:2]   
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, thealta, 1.0)
    rotated_img = cv2.warpAffine(dst, M, (w, h))
    if(top_bottom < 0):
        top_bottom = abs(top_bottom)
        rotated_img = rotated_img[top_bottom:-top_bottom, :, :]
    if(left_right < 0):
        left_right = abs(left_right)
        rotated_img = rotated_img[:, left_right:-left_right, :]
    return rotated_img
def make_new_bkg_img_by_eraser_objs(img_name, xml_name):
    print('----------------------1------------------------')
    #print(img_name)
    #print(xml_name)
    print('----------------------2------------------------')
    xml_file = open(xml_name)
    tree=ET.parse(xml_file)
    root = tree.getroot()
    bboxes = get_bboxes_from_etree(tree)
    img = cv2.imread(img_name)
    array = np.zeros_like(img)
    array.fill(200)
    img_objs = []
    for bbox in bboxes:
        img_obj = array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    return img

def generate_etree(etree_old):
    root_old = etree_old.getroot()
    
    root = ET.Element('annotation')
    root.append(root_old.find('folder'))
    root.append(root_old.find('filename'))
    root.append(root_old.find('source'))
    root.append(root_old.find('size'))
    root.append(root_old.find('segmented'))
    
    tree = ET.ElementTree(root)
    
    return tree

def generate_xml_and_image_from_miss_objs(seed_xml_file, s_objs, s_tree):
    img_name = seed_xml_file_.replace('/Annotations/', '/JPEGImages/')[:-3] + 'jpg'
    new_img = make_new_bkg_img_by_eraser_objs(img_name, seed_xml_file)
    new_tree = generate_etree(s_tree)
    for obj_element in s_objs:
        cls = get_cls_from_xmlobj(obj_element)
        
        bbox = get_bbox_from_xmlobj(obj_element)
        
        img_obj = get_obj_from_image_file(img_name, bbox)
        
        new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, new_img, obj_element, new_tree)
        if((new_pil_img == False) or (new_xml_etree == False)):
            continue
        new_tree = new_xml_etree
        new_img = np.array(new_pil_img)
        FLAG = True
    ## save xml and img
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
    new_pil_img = Image.fromarray(new_img[:,:,(2,1,0)])
    new_pil_img.save(GENE_IMG_DIR + time_str + img_name.split('/')[-1])
    new_tree.write(GENE_XML_DIR + time_str + img_name.split('/')[-1][:-3] + 'xml')

In [51]:
temp_list = [i.split('/')[-1] for i in seed_xml_names]
print('len temp_list : {0}'.format(len(temp_list)))
generated_xml_dir_father = '/media/yyp/DATA/EVALUATE/' + time_str + '_detection_result/'
generated_xml_dir_list = os.listdir(generated_xml_dir_father)
print(generated_xml_dir_list)
generated_xml_dir_list.sort()
cls_map = {'bus':'bus','car':'car', 'truck':'truck', 'B':'bus','C':'car', 
           'T':'truck', 'motorbike':'motorbike', 
           'bicycle':'bicycle', 'person':'person', 'plate':'plate'}

for generated_xml_dir in generated_xml_dir_list:
    print('-------------------------------------')
    print(generated_xml_dir)
    generated_xml_dir = generated_xml_dir_father + generated_xml_dir + '/'
    generated_xml_names = os.listdir(generated_xml_dir)
    generated_xml_names.sort()
    #print(generated_xml_dir)
    print('len generated_xml_names:{0}'.format(len(generated_xml_names)))
    total_obj_num = 0
    total_good_classify_num = 0
    total_erro_detected_num = 0
    total_miss_detected_num = 0
    total_good_detected_num = 0
    iou_thresh = 0.3

    for gene_xml in generated_xml_names:
        if(gene_xml in temp_list):
            #print(gene_xml + ' in temp_list')
            seed_xml_file_ = seed_xml_names[temp_list.index(gene_xml)]
            seed_xml_file = seed_xml_file_.replace('/JPEGImages/', '/Annotations/')
            gene_xml_file = generated_xml_dir + '/' + gene_xml
            print('----------seed_xml_file------------- \n' + seed_xml_file)

            #print(seed_xml_file)
            #print(gene_xml_file)

            s_tree=ET.parse(seed_xml_file)
            s_objs = get_obj_from_xml(seed_xml_file)
            ##s_bboxes = get_bboxes_from_etree(s_tree)

            g_tree=ET.parse(gene_xml_file)
            g_objs = get_obj_from_xml(gene_xml_file)
            ##g_bboxes = get_bboxes_from_etree(g_tree)

            detected_num = 0
            erro_bboxs = []
            total_obj_num += len(s_objs)
            #print(total_obj_num)
            #print('total obj number {0}'.format(len(s_objs)))
            #print('total detec number {0}'.format(len(g_objs)))
            '''
            for sobj in s_objs:            
            g_max_obj, iou_max = find_max_iou(sobj, g_objs)
            if(g_max_obj != False):
            print('max obj index:{0}'.format(g_objs.index(g_max_obj)))
            '''               
            for g_obj in g_objs:
                s_max_obj, iou_max = find_max_iou(g_obj, s_objs)
                if((s_max_obj != False) and (iou_max >= iou_thresh)):
                    #print('max obj index:{0}'.format(s_objs.index(s_max_obj)))
                    detected_num += 1
                    s_cls = get_cls_from_xmlobj(s_max_obj)
                    g_cls = get_cls_from_xmlobj(g_obj)
                    #print('s_cls:{0};g_cls:{1}'.format(s_cls, g_cls))
                    if(s_cls == cls_map[g_cls]):
                        total_good_classify_num += 1    
                    s_objs.remove(s_max_obj)                
                else:
                    erro_bboxs.append(g_obj)
            '''
            print("erro bbox number:{0}".format(len(erro_bboxs)))            
            print('good detected {0}'.format(detected_num))
            print('erro detected {0}'.format(len(erro_bboxs)))
            print('miss detected {0}'.format(len(s_objs)))
            '''
            total_erro_detected_num += len(erro_bboxs)
            total_miss_detected_num += len(s_objs)
            if(len(s_objs) > 0):
                generate_xml_and_image_from_miss_objs(seed_xml_file, s_objs, s_tree)
                #generate_xml_and_image_from_erro_objs(seed_xml_file,s_objs,s_tree)
            total_good_detected_num += detected_num
        else:
            #print(gene_xml + ' not in temp_list')
            pass
    print('good  : {0}'.format(total_good_detected_num))
    print('erro  : {0}'.format(total_erro_detected_num))
    print('miss  : {0}'.format(total_miss_detected_num))
    print('total : {0}'.format(total_obj_num))

    print('good rate : {0}'.format(total_good_detected_num/total_obj_num))
    print('erro rate : {0}'.format(total_erro_detected_num/total_obj_num))
    print('miss rate : {0}'.format(total_miss_detected_num/total_obj_num))
    print('clss rate : {0}'.format(total_good_classify_num/total_good_detected_num))


len temp_list : 100
['yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls-new-refine-shortcut_44000']
-------------------------------------
yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls-new-refine-shortcut_44000
len generated_xml_names:100
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011237726181_20191017142030461675_00010020109.xml
----------------------1------------------------
----------------------2------------------------
(26, 11, 3)
(26, 11, 3)
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011237838570_20191017142030486513_00010020114.xml
----------------------1------------------------
----------------------2------------------------
(29, 8, 3)
(29, 8, 3)
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqin

(78, 55, 3)
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011242230705_20191017142031478222_00010020177.xml
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011242326024_20191017142031502576_00010020178.xml
----------------------1------------------------
----------------------2------------------------
(61, 19, 3)
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011242421972_20191017142031529432_00010020179.xml
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011242517014_20191017142031554058_00010020190.xml
----------------------1------------------------
----------------------2---------

----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011251697386_20191017142033434436_00010020318.xml
----------------------1------------------------
----------------------2------------------------
(45, 16, 3)
(45, 16, 3)
(48, 13, 3)
(45, 16, 3)
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011251791828_20191017142033457344_00010020324.xml
----------------------1------------------------
----------------------2------------------------
(34, 10, 3)
(34, 10, 3)
(34, 10, 3)
(34, 10, 3)
(34, 10, 3)
(34, 10, 3)
----------seed_xml_file------------- 
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_chongqing_changanvehicle//VOC2007/Annotations/20191030011251884490_20191017142033480719_00010020328.xml
----------------------1------------------------
----------------------2-----------------